In [13]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
import torch.utils.data as td
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import math

froot = './proseqSimulator'

df = pd.read_csv(froot + ".csv")

Sequences were generated with: ./motifSimulator.py --m CTCF --N 2000 --len 500 --proseq 500,50000 --o proseqSimulator

In [14]:
ALPHABET = 'ACGT'
NALPH = len(ALPHABET)
INVALPH = [-1] * ord('Z')
for i, char in enumerate(ALPHABET):
    INVALPH[ord(char)] = i
        
def seq_to_one_hot(seq):
    seqlen = len(seq)
    res = np.zeros(NALPH * seqlen, dtype=np.uint8)    
    arr = np.array(list(seq))
    for j, c in enumerate(arr):
        res[NALPH*j + INVALPH[ord(c)]] = 1
    return res

In [15]:
# convert to X,Y pairs representing single-column counts (Y) 
# and corresponding centered 50bp segments of the sequence (X)
stride = 5
featlen = 50
offset = math.ceil(featlen/2)
seqlen = len(df['seq'][0])
centidx = range(offset, seqlen-offset, stride)

j = 0
y = np.zeros(len(centidx) * len(df),dtype=np.int16)
x = np.zeros((len(centidx)*len(df),4*featlen),dtype=np.uint8)

for i in range(len(df)):
    allcounts = np.array(df['readCounts'][i].strip('[]').split(),dtype=np.int16)  # better way?
    allseq = df['seq'][i]

    for r in centidx:
        y[j] = allcounts[r]
        x[j,:] = seq_to_one_hot(allseq[r-offset:r-offset+featlen])
        j += 1

In [16]:
# set them up as tensors
xtens = torch.Tensor(x)
ytens = torch.Tensor(y)

In [17]:
# split into train, test, and validation sets
allset = TensorDataset(xtens, ytens)
trnset, valset, tstset = td.random_split(allset, [0.5,0.25,0.25])

# set up data loaders
trndl = DataLoader(trnset, batch_size=128, shuffle=True)
tstdl = DataLoader(tstset, batch_size=128, shuffle=True)
valdl = DataLoader(valset, batch_size=128, shuffle=True)

In [18]:
# set up the model
import torch.nn as nn

# this model is slightly adapted from an image-processing CNN in 
#"Machine Learning with PyTorch and Scikit-Learn", Raschka et al.
model = nn.Sequential()
model.add_module(
    'conv1',
    nn.Conv1d(
        in_channels=1, out_channels=8,
        kernel_size=21, padding=10
    )
)
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool1d(kernel_size=21))

model.add_module(
    'conv2',
    nn.Conv1d(
        in_channels=8, out_channels=8,
        kernel_size=5, padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool1d(kernel_size=5))

model.add_module('flatten', nn.Flatten())
model.add_module('linear', nn.Linear(8,1))

# check model
print(model)

x = torch.ones((128,1,200))
print(model(x).shape)
nparm = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters: " + str(nparm))

Sequential(
  (conv1): Conv1d(1, 8, kernel_size=(21,), stride=(1,), padding=(10,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=21, stride=21, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(8, 8, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=8, out_features=1, bias=True)
)
torch.Size([128, 1])
Number of parameters: 513


In [19]:
# custom loss function
def my_loss(prediction, label):
#    loss = torch.mean((torch.exp(prediction) - label)**2)
    
    # poisson loss
    poismean = torch.exp(prediction)
    loss = torch.sum(-1*(-poismean + label*torch.log(poismean)))
    
    return loss

In [20]:
# debug the model
xbch, ybch = next(iter(trndl))
xsz = xbch.size()[0]
xbch_re = torch.reshape(xbch, (xsz, 1, 200))
pred = model(xbch_re)

print(my_loss(pred, ybch))

tensor(13879.6133, grad_fn=<SumBackward0>)


In [21]:
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}')
        model.train()
        for x_batch, y_batch in train_dl:
            # have to fix dimensionality
            x_batch_sz = x_batch.size()[0]
            x_batch_re = torch.reshape(x_batch, (x_batch_sz, 1, 200))
            pred = model(x_batch_re)    
            loss = my_loss(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            
        loss_hist_train[epoch] /= len(train_dl.dataset)
        
        model.eval()
        
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                # have to fix dimensionality
                x_batch_sz = x_batch.size()[0]
                x_batch_re = torch.reshape(x_batch, (x_batch_sz, 1, 200))
                pred = model(x_batch_re)    
                loss = my_loss(pred, y_batch)
                loss_hist_valid[epoch] += loss.item()*y_batch.size(0)
                
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            
        print(f'Epoch {epoch+1} trn_loss: '
              f'{loss_hist_train[epoch]:.4f} val_loss: '
              f'{loss_hist_valid[epoch]:.4f}')
        
    return loss_hist_train, loss_hist_valid

In [22]:
#loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, trndl, valdl)

Epoch 1
Epoch 1 trn_loss: 2491.1259 val_loss: 1736.0019
Epoch 2
Epoch 2 trn_loss: 2372.2029 val_loss: 1785.7658
Epoch 3
Epoch 3 trn_loss: 2359.0311 val_loss: 1805.0721
Epoch 4
Epoch 4 trn_loss: 2338.1486 val_loss: 1743.6646
Epoch 5
Epoch 5 trn_loss: 2353.3173 val_loss: 1707.1790
Epoch 6
Epoch 6 trn_loss: 2335.7142 val_loss: 1799.0806
Epoch 7
Epoch 7 trn_loss: 2313.9147 val_loss: 1703.2649
Epoch 8
Epoch 8 trn_loss: 2313.6792 val_loss: 1703.2776
Epoch 9
Epoch 9 trn_loss: 2300.8412 val_loss: 1765.1079
Epoch 10
Epoch 10 trn_loss: 2302.0450 val_loss: 1691.9772
Epoch 11
Epoch 11 trn_loss: 2316.4602 val_loss: 1698.3586
Epoch 12
Epoch 12 trn_loss: 2300.8581 val_loss: 1699.0755
Epoch 13
Epoch 13 trn_loss: 2298.0421 val_loss: 1708.5188
Epoch 14
Epoch 14 trn_loss: 2281.2957 val_loss: 1716.7742
Epoch 15
Epoch 15 trn_loss: 2292.0155 val_loss: 1736.8996
Epoch 16
Epoch 16 trn_loss: 2307.5026 val_loss: 1701.1357
Epoch 17
Epoch 17 trn_loss: 2286.5884 val_loss: 1684.3340
Epoch 18
Epoch 18 trn_loss: 2284